In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GridSearchCV, validation_curve, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, f1_score, plot_confusion_matrix, accuracy_score, \
    plot_roc_curve, plot_precision_recall_curve, classification_report, precision_score, recall_score
import warnings
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

В [задаче](https://www.kaggle.com/c/tabular-playground-series-apr-2021) рассматривается синтетический набор данных, созданный по мотивам [оригинального](https://www.kaggle.com/c/titanic) соревнования 'Титаник'. Данный 'дадасет' сгенерирован таким образом, что нет способа 'обмануть' модель, используя общедоступные метки, как это было в оригинальном соревновании.
Задача - предсказать, выжил ли пассажир после крушения 'Титаника'. Для каждого PasengerId, нужно предсказать значение 0 или 1 (бинарная классификация). Метрика используемая для оценки качества модели - Accuracy (доля верных ответов).
$$\large accuracy = \frac{TP + TN}{TP + TN + FP + FN}$$

# Features
***
|Переменная|Определение|Значения|
|:----------|:---------|:-----|
|Survived|Выживание|(0 = Нет, 1 = Да)|
|Pclass|Класс билета|(1 = 1st, 2 = 2nd, 3 = 3rd)|
|Sex|Пол|
|Age|Возраст в годах|
|SibSp|# Братьев и сестер/супругов на борту "Титаника"|
|Parch|# Родителей/детей на борту "Титаника"|	
|Ticket|Номер билета|
|Fare|Пассажирский тариф (стоимость)|  	
|Cabin|Номер каюты|
|Embarked|Порт отправления|(C = Cherbourg, Q = Queenstown, S = Southampton)|

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')
submission = pd.read_csv('../input/tabular-playground-series-apr-2021/sample_submission.csv')
# объединение test и train
data = train.append(test, ignore_index=True)

# Exploratory data analysis

## 1. Data visualization: First Overview

In [ ]:
train.info()

В дадасете есть как числовые, так и категориальные признаки, также есть пропущенные значения.

## 1.1 Target Variable

In [ ]:
# seaborn style
sns.set_style("whitegrid")
sns.despine(left=True, bottom=True)
sns.set_context("notebook")
plt.rcParams["figure.figsize"] = (16,6)

In [ ]:
ax = sns.countplot(y='Survived', data=train, alpha=0.8)
total = train.shape[0]

for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_width() / total)
    x = p.get_x() + p.get_width()
    y = p.get_y() + p.get_height() / 2
    ax.annotate(percentage, (x, y))

Классы в тренировачной выборке можно считать сбалансированными.

## 1.2 Data missings

In [ ]:
# строим таблицу пропущенных значений в данных
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(11)

Из таблицы видно, что признак 'Cabin' имеет почти 70% пропущенных значений. В остальных признаках, пропуски не так критичны.

## 1.3 Numerical features
Посмотрим на распределение вещественных признаков, в нашем случае это 'Age' и 'Fare'.


In [ ]:
sns.displot(data=train, x='Age', hue='Survived', kde=True, height=5, aspect=3)
sns.displot(data=train, x='Fare', hue='Survived', log_scale=2, kde=True, height=5, aspect=3);

* На графике распределения возраста видно, что молодые люди ~ от 18 до 42 лет погибали чаще чем другие. Выживаемость среди пожилых, напротив, была выше. 
* Логарифмированный график переменной 'Fare' указывает на прямую зависимость суммы потраченой на билет и выживаемости. Пассажиры заплатившие больше 100$ выживали чаще остальных.

## 1.4 Categorical features

In [ ]:
# определяем список категориальных переменных для которых будут строиться графики
cat_cols = ['Survived', 'Sex', 'Pclass', 'SibSp', 'Parch', 'Embarked']
# определяем кол-во столбцов, строк и создаем subplot заданного размера   
rows = 2
cols = 3
fig, axs = plt.subplots(rows, cols, figsize = (20,10))
# в цикле строим графики для каждой переменной
for i in range(rows):
    for j in range(cols):
        n = i*cols+j
        ax=axs[i][j]
        sns.countplot(data=train, x=cat_cols[n], hue='Survived', alpha=0.8, ax=ax)
        ax.set_title(cat_cols[n], fontsize=18, fontweight='bold')
        ax.legend(title="survived", loc='upper center')
plt.tight_layout()

* Sex: мужнин было больше и погибали они с большей вероятностью чем женщины.
* Psclass: У пассажиров первого класса было больше шансов выжить, люди из 3-го класса погибали чаще остальных.
* SibSp, Parch: Обе переменные указывают на кол-во родственников на борту корабля для каждого пассажира, можно сделать вывод, что путешествовать в одиночку было безопаснее. 
* Embraked: У пассажиров отправившихся из порта Cherbourg было больше всего шансов выжить.

## 1.5 Correlation matrix

In [ ]:
# импорт библиотеки для построения корреляционных матриц
# https://phik.readthedocs.io/en/latest/
import phik
from phik import resources, report

In [ ]:
train_for_matrix = train.drop(['PassengerId', 'Name', 'Ticket'], axis=1)
thik = train_for_matrix.phik_matrix()

In [ ]:
plt.figure(figsize=(10, 10))
mask = np.triu(np.ones_like(thik, dtype=np.bool))
sns.heatmap(thik, square=True, annot = True, center=0, vmax=.8, cmap='viridis', linewidths=1, linecolor='black', cbar=False, mask=mask);

* Целевая переменная 'Survived' сильнее всего коррелирует с переменной 'Sex', что подтверждает предположение о том, что женщины выживали с большей вероятностью.
* Самая сильная корреляция между 'Pclass' и 'Cabin', эту зависимость рассмотрим позже.# 2. Data visualization: Compare Train and Test
Проведем сравнение и построим графики для train и test выборок.

# 2. Data visualization: Compare Train and Test
Проведем сравнение и построим графики для train и test выборок.

## 2.1 Categorical Features

In [ ]:
# объединяем Train и Test с сохранением метки к какому набору данных изначально принадлежал объект
titaniс_visual = pd.concat([train.assign(dataset='train'), test.assign(dataset='test')])

In [ ]:
# определяем список категориальных переменных для которых будут строиться графики
cat_cols = ['Sex', 'Pclass', 'SibSp', 'Parch', 'Embarked', 'Survived']
# определяем кол-во столбцов, строк и создаем subplot заданного размера   
rows = 2
cols = 3
fig, axs = plt.subplots(rows, cols, figsize = (20,10))
# в цикле строим графики для каждой переменной
for i in range(rows):
    for j in range(cols):
        n = i*cols+j
        ax=axs[i][j]
        sns.countplot(data=titaniс_visual, x=cat_cols[n], hue='dataset', alpha=0.8, ax=ax)
        ax.set_title(cat_cols[n], fontsize=18, fontweight='bold')
        ax.legend(title="dataset", loc='upper center')
plt.tight_layout()

Размер выборок одинаковый, в обоих датасетах по 100.000 объектов.
* Sex: мужчины доминируют и в train и в test, есть лишь небольшая разница в пропорции.
* Pclass: Соотношение переменной Pclass одинаково, больше всего пассажиров путишевствовали 3-м классом, меньше 2-м. В тестовой выборке сильно меньше объектов 2-го класса.
* SibSp, Parch: в целом, значения обоих переменных распределены раномерно.

## 2.2 Numerical features

In [ ]:
sns.displot(data=titaniс_visual, x='Age', hue='dataset', kde=True, height=5, aspect=3)
sns.displot(data=titaniс_visual, x='Fare', hue='dataset', log_scale=2, kde=True, height=5, aspect=3);

* Из сравнения распределений переменной 'Age' по двум выборкам, видно, что в test намного больше молодых людей ~ от 15 до 30 лет, в то-же время, в test меньше взрослых людей ~ от 40 до 80 лет. 
* Сравнительный график 'Fare' показывает, что в тестовом наборе данных больше людей заплативших больше 100$ за билет.

# 3. Feature Engineering


## 3.1 Fill empty
Заполним пропуски в каждой переменной

In [ ]:
# строим таблицу пропущенных значений в данных
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(11)

### Cabin
Больше всего пропусков в признаке Cabin, порядка 70% от общего кол-ва объектов. Из предыдущего соревнования известно, что в номере каюты "зашито" буквенное обозначение палубы. То, на какой палубе находилась каюта имело влияние на выживаемость. 

In [ ]:
# срезаем первый символ от каждого заполненного значения, это будет номер(название) палубы
data['Deck'] = data['Cabin'].str[0]
# для незаполненных значений создаем новую категорию 'N'
data['Deck'] = data['Deck'].fillna('N')
# Cabin был заменен на Deck
data.drop('Cabin', axis=1, inplace=True)

In [ ]:
# смотрим на результаты
sns.countplot(x="Deck", hue='Survived', data=data);
data['Deck'].value_counts()

### Age
Поскольку корреляция между 'Age' и 'Pclass' довольно высока, заполним пропущенные значения - средним значением, для каждого пола в каждом классе.

In [ ]:
# группируем по двум признакам
grp = data.groupby(['Sex', 'Pclass'])
# заполняем пропуски средним для каждой группы
data['Age'] = grp['Age'].apply(lambda x: x.fillna(x.mean()))

### Fare
У признака 'Fare' всего 267 пропущенных значений в общей выборке. Заполнять будем медианным значением для каждого 'Pclass'.

In [ ]:
# группируем 
grp = data.groupby(['Pclass'])
# заполняем пропуски медианой для каждой группы
data['Fare'] = grp['Fare'].apply(lambda x: x.fillna(x.median()))

### Embarked
У Embarked 527 пропущенных значений, поскольку наблюдается сильный дисбаланс в пользу значения 'S', заполним пропуски им.

In [ ]:
data['Embarked'] = data['Embarked'].fillna('S')

### Ticket
Примерно 5% пропущенных значений, зависимости от других переменных установить не удалось. Похоже, цифры в номере билета не нусут никакой информации, поэтому оставляем только буквенный индекс перед номером.

In [ ]:
# от существующих значений берем сплит до первого пробела, если нет, то заполняем 'X'
data['Ticket'] = data['Ticket'].map(lambda x:str(x).split()[0] if len(str(x).split()) > 1 else 'X')

In [ ]:
data['Ticket'].value_counts().head(10)

## 3.2 Add new Features

### Family
Признаки SibSp, Parch говорят об одном и том же, колличестве родственников на борту, поэтому объединяем два признака в один.

In [ ]:
data['Family'] = data['Parch'] + data['SibSp']

### Alone
Людей путешествующих в одиночку можно выделить в отдельный признак.

In [ ]:
data['Alone'] = data['Family'] == 0

### FareC
Преобразуем вещественную переменную 'Fare' в категориальную.

In [ ]:
# задаем интервалы
bins = [0,10,20,25,30,65,80,110,745]
# создаем список меток для каждого интервала
lables = ['FareC-' + str(i) for i in range(len(bins)-1)]
# превращаем вещественную переменную Fare в категориальную FareC
data['FareC'] = pd.cut(data['Fare'], bins=bins, labels=lables)

In [ ]:
sns.countplot(x="FareC", hue='Survived', data=data);

### Age
Так же преобразуем переменную 'Age' в категориальную 'AgeC'.

In [ ]:
# задаем интервалы
bins = [0,5,10,20,30,40,50,60,70,80,90]
# создаем список меток для каждого интервала
lables = ['AgeC-' + str(i) for i in range(len(bins)-1)]
# превращаем вещественную переменную Fare в категориальную FareC
data['AgeC'] = pd.cut(data['Age'], bins=bins, labels=lables)

In [ ]:
sns.countplot(x="AgeC", hue='Survived', data=data);

### Name
Оставим только фамилии, срезав имена

In [ ]:
data['Name'] = data['Name'].apply(lambda x: str(x).split(',')[0])

In [ ]:
# смотрим на 10 самых популярных фамилий
data['Name'].value_counts().head(10)

In [ ]:
data.info()

# 4. Model

In [ ]:
# датафрейм для записи результатов
report_df = pd.DataFrame()

def get_scores(report_df, model, X_test, y_test, name):
    '''Принимает на вход датафрейм, обученную модель и данные с отложенной выборки, 
    рассчитывает метрики и записывает результаты в датафрейм'''
    report = pd.DataFrame(columns={'ROC-AUC'}, data=[0])
    report['ROC-AUC'] = roc_auc_score(y_test,
                                      model.predict_proba(X_test)[:, 1])
    report['F1'] = f1_score(y_test, model.predict(X_test))
    report['precision_0'] = precision_score(
        y_test, model.predict(X_test), pos_label=0)
    report['precision_1'] = precision_score(
        y_test, model.predict(X_test), pos_label=1)
    report['recall_0'] = recall_score(
        y_test, model.predict(X_test), pos_label=0)
    report['recall_1'] = recall_score(
        y_test, model.predict(X_test), pos_label=1)
    report['accuracy'] = accuracy_score(
        y_test, model.predict(X_test))
    report.index = [name]
    report_df = report_df.append(report)
    return report_df

## 4.1 Features selection & encoding

In [ ]:
label_cols = ['Name', 'Ticket', 'Family', 'Alone']
onehot_cols = ['Pclass', 'Sex', 'Deck', 'Embarked']
numerical_cols = ['Age', 'Fare', 'Survived']

In [ ]:
def label_encoder(c):
    lc = LabelEncoder()
    return lc.fit_transform(c)

In [ ]:
onehot_encoded_df = pd.get_dummies(data[onehot_cols])
label_encoded_df = data[label_cols].apply(label_encoder)
numerical_df = data[numerical_cols]

In [ ]:
data = pd.concat([numerical_df, label_encoded_df, onehot_encoded_df], axis=1)

In [ ]:
data.info()

## 4.2 Split data

In [ ]:
# разделяем общий дадасет data на train и test
# в тестовый датасет попадают только те объекты для которых значение 'Survived' не определено
predict_data = data[data['Survived'].isnull()]
predict_data = predict_data.drop(['Survived'], axis = 1)

# в тренировачный набор попадают объекты у которых нет пропущеных значений ни у одной переменной
train_data = data.dropna()
target = train_data['Survived']
train_data = train_data.drop(['Survived'], axis = 1)
# контроль разммера
train_data.shape

# Random Forest model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier# разделяем на отложенную и ообучающую
X_train, X_test, y_train, y_test = train_test_split(train_data, target, 
                                                    test_size=0.2, 
                                                    random_state=23)

In [ ]:
rf = RandomForestClassifier(oob_score=True)
rf_params = {
    'n_estimators': [3000],
    'min_samples_split': [2,3,5,7],
    'min_samples_leaf': [8,10,12]
}
# подбор гиперпараметров
rf_rand = RandomizedSearchCV(rf, rf_params, cv=3, verbose=5, n_jobs=-1, n_iter=5, random_state=54, scoring = "accuracy")
rf_rand.fit(X_train, y_train)

In [ ]:
# лучший score и лучшие параметры на кросс-валидации
print("Accuracy (random forest auto): {} params {}"
      .format(rf_rand.best_score_, rf_rand.best_params_))
# записываем результаты на отложенной выборке в датафрейм
report_df = get_scores(report_df, rf_rand, X_test,
                       y_test, 'Random_Forest')

In [ ]:
report_df

# Logistic Regression model


In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data, target, 
                                                    test_size=0.2, 
                                                    random_state=23)

In [ ]:
lr = LogisticRegression()
lr_params = {
    'C': np.linspace(0.001, 10, 20), 
    'penalty': ['l1', 'l2']
}
# подбор гиперпараметров
lr_rand = RandomizedSearchCV(lr, lr_params, cv=5, verbose=5, n_jobs=-1, n_iter=20, random_state=54, scoring = "accuracy")
lr_rand.fit(X_train, y_train)

In [ ]:
# лучший score и лучшие параметры на кросс-валидации
print("Accuracy (random forest auto): {} params {}"
      .format(lr_rand.best_score_, lr_rand.best_params_))
# записываем результаты на отложенной выборке в датафрейм
report_df = get_scores(report_df, lr_rand, X_test,
                       y_test, 'Logistic_Regression')

In [ ]:
report_df

# LightGBM model

In [ ]:
import lightgbm as lgb
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data, target, 
                                                    test_size=0.2, 
                                                    random_state=23)

In [ ]:
lgbc = lgb.LGBMClassifier()
lgbc_params = {
             'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=2), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100],
             'scale_pos_weight':[1,2,6,12],
             'metric':'binary_logloss',
             'n_estimators': [100, 250, 500, 1000]
}
lgbc_rand = RandomizedSearchCV(lgbc, lgbc_params, cv=5, verbose=5, n_jobs=-1, n_iter=20, random_state=54, scoring = 'accuracy')
lgbc_rand.fit(X_train, y_train)

In [ ]:
# лучший score и параметры на кросс-валидации
print("Accuracy (random forest auto): {} params {}"
      .format(lgbc_rand.best_score_, lgbc_rand.best_params_))
# записываем результаты на отложенной выборке в датафрейм
report_df = get_scores(report_df, lgbc_rand, X_test,
                       y_test, 'LGBM')

In [ ]:
report_df

# XGBoost

In [ ]:
import xgboost as xgb
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data, target, 
                                                    test_size=0.2, 
                                                    random_state=23)

In [ ]:
xgbc = xgb.XGBClassifier( tree_method = "exact", predictor = "cpu_predictor", verbosity = 1,
                           eval_metric = "logloss", objective = "binary:logistic")
# Create parameter grid
xgbc_params = {
               "learning_rate": [0.01, 0.05, 0.1, 0.15, 0.2],
               "gamma" : [0.3, 0.5, 1, 1.5, 2, 3, 4],
               "max_depth": [3, 4, 5, 6, 10],
               "colsample_bytree": sp_uniform(loc=0.2, scale=1),
               "subsample": sp_uniform(loc=0.2, scale=2),
               "reg_alpha": [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
               "reg_lambda": [0, 1e-1, 1, 5, 10, 20, 50, 100],
               "min_child_weight": [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
               "n_estimators": [100, 250, 500, 1000]
}

# Create RandomizedSearchCV Object
xgbc_rand = RandomizedSearchCV(xgbc, param_distributions = xgbc_params, scoring = "accuracy",
                             cv = 5, verbose = 3, random_state = 40)
# Fit the model
model_xgboost = xgbc_rand.fit(X_train, y_train)

In [ ]:
# лучший score и параметры на кросс-валидации
print("Accuracy (random forest auto): {} params {}"
      .format(xgbc_rand.best_score_, xgbc_rand.best_params_))
# записываем результаты на отложенной выборке в датафрейм
report_df = get_scores(report_df, xgbc_rand, X_test,
                       y_test, 'XGB')

In [ ]:
report_df

# Feature importances

In [ ]:
xgbc_ = xgb.XGBClassifier(eval_metric = "logloss", objective = "binary:logistic", 
                          colsample_bytree = 0.7083099649576006, gamma = 2, learning_rate = 0.05, 
                          max_depth = 6, min_child_weight = 0.1, n_estimators = 500, reg_alpha = 1, 
                          reg_lambda = 100, subsample = 0.7675145143612605)
xgbc_.fit(X_train, y_train)

In [ ]:
plt.rcParams["figure.figsize"] = (24,8)

sorted_idx = xgbc_.feature_importances_.argsort()
plt.barh(X_train.columns[sorted_idx], xgbc_.feature_importances_[sorted_idx], ecolor='g')
plt.xlabel("Xgboost Feature Importance");


In [ ]:
# делаем прогноз лучшей моделью
pred = xgbc_.predict_proba(predict_data)

In [ ]:
# отправляем посылку на Кегл
submission['Survived'] = ((pred) < 0.5).astype(int)
submission.to_csv('predict_xgb.csv', index = False)